In [1]:
from IPython.core.display import display, HTML
display(HTML('<p style="text-align:center;"><img src="https://www.shareicon.net/data/512x512/2017/05/22/886198_twitter_512x512.png" alt="Twitter Logo"></p>')) 

# <div align='center'>Sentiment Analysis of Twitter Data</div>
## <div align='center'>The New and Improved(?) Twitter API 2.0</div>

Documentation:  
[Complete Walk-thru on my Blog](https:www.girlexmachina.com)  
[YouTube Playlist for all Parts]()  
[Twitter API](https://developer.twitter.com/en/docs/developer-portal/overview)  

## Background
As an application, Twitter has opened up access to opinions and ideas like no other form of social media.  You can follow, tag, and mention anyone with an account to allow others to hear your views and opinions.  As of late, Twitter has been used by many activist to express their concerns to members of the media, government, and corporate america.  The true question is do companies utilize Twitter data for marketing feedback?  It's virtually free and can be a good thermomater for the feelings of the online crowd.

<mark>I think it should go without saying that this is in no way a beginner tutorial.</mark>  Maneuvering the backwaters and bayous of the official Twitter documentation can be a difficult task, even for those who are already initiated.  It's possible that this code will run if you simply have a bearer token.  If you don't know what that is, then I strongly suggest you follow the walk-thru posted to my blog, and definitely the video series.  Otherwise you could be more confused than when you started.

## Overview
In this tutorial, we will analyze tweets before and after a movie trailer release to see if there was positive or negative sentiment towards the movie.  We will then compare our findings with box office numbers to see if our algorithm is accurate!  Scraping the tweets is the most difficult task in this tutorial.  Twitter severely limits the rate at which you can pull tweets (per 15 minutes), the total number of tweets, and also (to prevent the overwhelming of their API, force you to code for pagenation at 10 tweets per request.  As I'm using one of my many Twitter dev accounts, I'm going to try to pull as many extended tweets as possible.  Let's get started!

In [1]:
# Standard Libraries
import os, requests, json
import pandas as pd
import tweepy as tp

In [2]:
from config import twit_access_secret, twit_access_token, twit_api_key, twit_api_secret_key, twit_bearer_token

In [3]:
os.makedirs("scraped_tweets/",exist_ok=True)

## Authentication

In [4]:
# This code block is from the official documentation (with a minor, personal touch!)
auth = tp.OAuthHandler("twit_api_key", "twit_api_secret_key")
auth.set_access_token("twit_access_token", "twit_access_secret")
try:
    twitter = tp.API(auth)
    twitter.verify_credentials()
    print('Giddy-up!')
except:
    print('Check your settings!')

Giddy-up!


In [5]:
i=0
USER_ID = 15687962
bearer_token = twit_bearer_token

## Harvest Tweets

In [8]:
user_id = USER_ID
url = "https://api.twitter.com/2/users/{}/mentions".format(user_id)

EXPANSIONS = 'author_id,referenced_tweets.id,referenced_tweets.id.author_id,in_reply_to_user_id,attachments.media_keys,attachments.poll_ids,geo.place_id,entities.mentions.username'
MEDIA_FIELDS = 'duration_ms,height,media_key,preview_image_url,type,url,width,public_metrics'
TWEET_FIELDS = 'created_at,author_id,public_metrics'
USER_FIELDS = 'location,profile_image_url,verified'

params =  {'max_results':100,'expansions': EXPANSIONS,'tweet.fields': TWEET_FIELDS,'user.fields': USER_FIELDS,'media.fields': MEDIA_FIELDS}
headers = {"Authorization": "Bearer {}".format(bearer_token)}

In [9]:
response = requests.request("GET", url, headers=headers, params=params)
print(response.status_code)
if response.status_code != 200:
    raise Exception(
        "Request returned an error: {} {}".format(
            response.status_code, response.text
        )
    )

200


In [10]:
json_response = response.json()

In [11]:
df_tweets = pd.DataFrame(json_response['data'])
df_meta = json_response['meta']
df_tweets.to_csv(f'./scraped_tweets/marvel{i:02d}.csv')

In [12]:
df_meta

{'oldest_id': '1406063444789547008',
 'newest_id': '1406098096212357124',
 'result_count': 100,
 'next_token': '7140dibdnow9c7btw3z0s077l00l7t8iprf8u7odj554o'}

In [13]:
url = "https://api.twitter.com/2/users/{}/mentions".format(user_id)
for i in range(1,20):
    token = json_response['meta']['next_token']
    params['pagination_token']=json_response['meta']['next_token']
    response = requests.request("GET", url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    else:
        json_response = response.json()
        df_tweets = pd.DataFrame(json_response['data'])
        df_meta = json_response['meta']
        df_tweets.to_csv(f'./scraped_tweets/marvel{i:02d}.csv')
        print(f'Round {i} saved!')

Round 1 saved!
Round 2 saved!
Round 3 saved!
Round 4 saved!
Round 5 saved!
Round 6 saved!
Round 7 saved!
Round 8 saved!


KeyError: 'next_token'